# 环境初始化

In [ ]:
import aie
import webbrowser

from aiearth.core.error import aie_error

import os
mytoken = os.environ.get('aiearth_token')

if not mytoken:
    mytoken = "your token here"

# 鉴权失败会打开手册
try:
    aie.Authenticate(token=mytoken)
    aie.Initialize()
except AttributeError as e:
    print(e)
    print('aie-sdk可能没有被正确安装！')
except aie_error as e:
    print(e)
    print('无效的token！')
    webbrowser.open("https://engine-aiearth.aliyun.com/#/utility/auth-token")
except :
    webbrowser.open('https://engine-aiearth.aliyun.com/docs/page/api?d=07f36f#heading-22:~:text=%E7%BC%96%E5%86%99%E4%BB%A3%E7%A0%81-,%E9%89%B4%E6%9D%83%E5%8F%8A%E5%88%9D%E5%A7%8B%E5%8C%96,-%E6%82%A8%E5%8F%AF%E4%BB%A5%E5%9F%BA%E4%BA%8E')

# 确定roi

In [ ]:

feature_collection_1 = aie.FeatureCollection('China_Province') \
                        .filter(aie.Filter.eq('province', '广西壮族自治区'))

feature_collection_2 = aie.FeatureCollection('China_Province') \
                        .filter(aie.Filter.eq('province', '广东省'))

union_fc = feature_collection_1.merge(feature_collection_2).union()
union_geometry = union_fc.geometry()

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

vis_params = {
    'color': '#00FF00'
}

map.addLayer(
    union_fc,
    vis_params,
    'region',
    bounds=union_fc.getBounds()
)
map

# aie.Export.feature.toAsset(union_fc, "union_fc").start()

# 辅助函数

In [ ]:
from math import sqrt


def get_max_and_min(img_target:aie.Image,region:aie.Geometry=union_geometry):
    reducer = aie.Reducer.max().combine(reducer2=aie.Reducer.min(), sharedInputs=True)
    result = img_target.reduceRegion(reducer, region)
    max,min = list(result.getInfo().values())
    return max,min

def fix_with_mean(img_target:aie.Image,region:aie.Geometry=union_geometry):
    reducer = aie.Reducer.mean()
    result = img_target.reduceRegion(reducer, union_geometry)

    mean_tag = list(result.getInfo().keys())[0]
    target_mean = result.getInfo()[mean_tag] 
    
    img_mean = aie.Image(target_mean).clip(union_geometry)
    img_target = img_target.unmask(img_mean)
    img_target = img_target.updateMask(img_target.mask())
    
    return img_target


def save(img_target:aie.Image, filename:str=None):
    if filename is None:
        filename = img_target.bandNames().getInfo()[1]
    
    # 获取分辨率
    reducer = aie.Reducer.min()
    result = img_target.pixelArea().reduceRegion(reducer, union_geometry)
    area = sqrt(result.getInfo()['area_min'])
    
    name = img_target.bandNames().getInfo()[0]
    print(name + ":" + str(area))

    # 导出到持久化空间
    # aie.Export.image.toAsset(img_target, filename, area).start()
        

# 获取数据集

In [ ]:
# 月度火烧迹地产品（MCD64A1 v006）
# https://engine-aiearth.aliyun.com/#/dataset/MODIS_MCD64A1_006

dataset_firepoint = aie.ImageCollection('MODIS_MCD64A1_006') \

# 用or运算提取所有火点
img_firepoint = dataset_firepoint.select(['BurnDate']).Or() \
                            .mask() \
                            .clip(union_geometry) \
                            .rename(["firepoint"])


vis_params = {
    'palette': ["0,0,0","255,0,0"],
    'min': 0,
    'max': 1,
}

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_firepoint,
    vis_params,
    "firepoint",
    bounds=union_fc.getBounds()
)

map

# save(img_firepoint,"img_firepoint")

In [ ]:
# DEM
# https://engine-aiearth.aliyun.com/#/dataset/Copernicus_DEM_30M
dataset_DEM = aie.ImageCollection('Copernicus_DEM_30M') \
            .filterBounds(union_fc) \
            .select(['elevation'])

img_DEM = dataset_DEM.mosaic().clip(union_geometry)

# 坡度和坡向
img_slope = aie.Terrain.slope(img_DEM).rename(["slope"])
img_aspect = aie.Terrain.aspect(img_DEM).rename(["aspect"])


vis_params = {
    'min': 0,
    'max': 500,
    'palette': [
        '0,0,255','0,255,255','0,255,0','255,255,0','255,0,0'
    ]
}

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_DEM,
    vis_params,
    "DEM",
    bounds=union_fc.getBounds()
)

map

# save(img_DEM, 'dem')
# save(img_slope, 'img_slope')
# save(img_aspect, 'img_aspect')

In [ ]:
# 地物分类
# https://engine-aiearth.aliyun.com/#/dataset/ESA_WORLD_COVER_V200

dataset_classify = aie.ImageCollection('ESA_WORLD_COVER_V200') \
             .filterBounds(union_geometry)

img_cover = dataset_classify.mosaic() \
                            .clip(union_geometry) \
                            .rename(["cover"])




vis_params = {
    'SR_Bands': 'cover',
    'min': 10.0,
    'max': 100.0,
    'palette': [
        '#006400','#ffbb22','#ffff4c','#f096ff',
        '#fa0000','#b4b4b4','#f0f0f0','#0064c8',
        '#0096a0','#00cf75','#fae6a0'
    ]
}

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_cover,
    vis_params,
    "cover",
    bounds=union_fc.getBounds()
)

map

# save(img_cover,"img_cover")

In [ ]:
dataset_ndvi = aie.ImageCollection('MODIS_MOD13Q1_061') \
             .filterDate('2021-01-01', '2021-12-31') \
             .select(['NDVI']) 

img_ndvi = dataset_ndvi.mean() \
                        .clip(union_geometry) \
                        .rename(["NDVI"])


vis_params = {
    'SR_Bands': 'NDVI',
    'min': 0,
    'max': 8000,
    'palette': [ 
        '#FFFFFF', '#CE7E45', '#DF923D', '#F1B555', '#FCD163', '#99B718',
        '#74A901', '#66A000', '#529400', '#3E8601', '#207401', '#056201',
        '#004C00', '#023B01', '#012E01', '#011D01', '#011301'
    ]
}


map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_ndvi,
    vis_params,
    "NDVI",
    bounds=union_fc.getBounds()
)

map

# save(img_ndvi,"img_ndvi")

In [ ]:
# ERA5-Land monthly averaged data
# https://engine-aiearth.aliyun.com/#/dataset/ERA5_LAND_MONTHLY
# 月度生物气候变量

dataset_Bioclimatic = aie.ImageCollection('ERA5_LAND_MONTHLY') \
                        .filterDate('2021-01-01', '2021-12-31') \
                        .filterBounds(union_fc)

# 平均2m温度             
dataset_avg_temp = dataset_Bioclimatic.select(['temperature_2m'])
img_avg_temp = dataset_avg_temp.mean().clip(union_geometry).rename(["avg_temp"])
# 平均地表温度
dataset_skin_avg_temp = dataset_Bioclimatic.select(['skin_temperature'])
img_skin_avg_temp = dataset_skin_avg_temp.mean().clip(union_geometry).rename(["skin_avg_temp"])
# 平均气压
dataset_surface_pressure = dataset_Bioclimatic.select(['surface_pressure'])
img_surface_pressure = dataset_surface_pressure.mean().clip(union_geometry).rename(["surface_pressure"])
# 年降水量
dataset_precipitation = dataset_Bioclimatic.select(['total_precipitation'])
img_precipitation = dataset_precipitation.sum().clip(union_geometry).rename(["precipitation"])
# 年蒸发量
dataset_evaporation = dataset_Bioclimatic.select(['total_evaporation'])
img_evaporation = dataset_evaporation.sum().clip(union_geometry).rename(["evaporation"])


# save(img_surface_pressure,"img_surface_pressure")
# save(img_skin_avg_temp,"img_skin_avg_temp")
# save(img_precipitation,"img_precipitation")
# save(img_evaporation,"img_evaporation")

In [ ]:
# WET
# 湿度指标
dataset = aie.ImageCollection('LANDSAT_LC09_C02_T1_L1') \
             .filterBounds(union_fc) \
             .filterDate('2021-01-01', '2022-12-31') \
             # .filter(aie.Filter.lte('eo:cloud_cover', 30.0))

# 去云算法
# https://engine-aiearth.aliyun.com/docs/page/case?d=6cf4e8
def removeLandsatCloud(image):
    # cloudShadowBitMask = (1 << 4)
    cloudsBitMask = (1 << 3)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(aie.Image(cloudsBitMask)).eq(aie.Image(0))
    return image.updateMask(mask)

# def applyScaleFactors(image):
#     opticalBands = image.select('SR_B.').multiply(aie.Image(0.0000275)).add(aie.Image(-0.2))
#     thermalBands = image.select('ST_B.*').multiply(aie.Image(0.00341802)).add(aie.Image(149.0))
#     return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

# images_no_cloud = dataset.map(removeLandsatCloud).map(applyScaleFactors)
images_no_cloud = dataset.map(removeLandsatCloud)

image_L9C2L2 = images_no_cloud.mosaic().clip(union_geometry)

# WET = 0.1511 B1 + 0.1973 B2 + 0.3283 B3 + 0.3407 B4 - 0.7171 B5 - 0.4559 B6
# http://www.yndxxb.ynu.edu.cn/yndxxbzrkxb/article/doi/10.7540/j.ynu.20190174?viewType=HTML
# https://zhuanlan.zhihu.com/p/479851365

# L9C2L1
img_wet = image_L9C2L2.select('B2').multiply(aie.Image(0.1511)) \
    .add(image_L9C2L2.select('B3').multiply(aie.Image(0.1973))) \
    .add(image_L9C2L2.select('B4').multiply(aie.Image(0.3283))) \
    .add(image_L9C2L2.select('B5').multiply(aie.Image(0.3407))) \
    .add(image_L9C2L2.select('B6').multiply(aie.Image(0.7171))) \
    .add(image_L9C2L2.select('B7').multiply(aie.Image(0.4559))) \
    .rename(["wet"])

img_wet = fix_with_mean(img_wet)

max,min = get_max_and_min(img_wet)

vis_params = {
    'min': min,
    'max': max,
    'palette': [ 
        '255,255,0', '0,0,255',
    ]
}

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_wet,
    vis_params,
    "WET",
    bounds=union_fc.getBounds()
)

map

# save(img_wet, 'wet')

In [ ]:
# WorldPop Estimated Residential Population
# https://engine-aiearth.aliyun.com/#/dataset/WORLDPOP_GP_100M_POP

dataset_pop = aie.ImageCollection('WORLDPOP_GP_100M_POP') \
                .filterDate('2021-01-01','2021-12-31') \
                .filterBounds(union_geometry) \
                .select(['population'])


img_pop = dataset_pop.mosaic().clip(union_geometry)

vis_params = {
      'bands': ['population'],
      'min': 0.0,
      'max': 50.0,
      'palette': ['#24126c', '#1fff4f', '#d4ff50']
}

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_pop,
    vis_params,
    "pop",
    bounds=union_fc.getBounds()
)

map

# save(img_pop, 'population')

In [ ]:
# OpenLandMap Soil Texture Class
# https://engine-aiearth.aliyun.com/#/dataset/OPENLANDMAP_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_V02

img_soil = aie.Image('OPENLANDMAP_SOL_SOL_TEXTURE-CLASS_USDA-TT_M_V02')
                
img_soil = img_soil.select(['b0']).clip(union_geometry).rename(["soil"])

vis_params = {
    'bands': 'soil',
    'min': 1.0,
    'max': 12.0,
    'palette': [
        "#d5c36b","#b96947","#9d3706","#ae868f","#f86714","#46d143",
        "#368f20","#3e5a14","#ffd557","#fff72e","#ff5a9d","#ff005b",
  ]
}

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_soil,
    vis_params,
    "soil",
    bounds=union_fc.getBounds()
)

map

# save(img_soil,"img_soil")

# 自适应分类模型

In [ ]:
# 采样

label = "firepoint"
img_firepoint = img_firepoint.rename([label])

img_all = img_firepoint.addBands(img_aspect) \
                        .addBands(img_slope) \
                        .addBands(img_DEM)  \
                        .addBands(img_pop)  \
                        .addBands(img_precipitation) \
                        .addBands(img_cover) \
                        .addBands(img_soil) \
                        .addBands(img_surface_pressure) \
                        .addBands(img_avg_temp) \
                        .addBands(img_ndvi) \
                        .addBands(img_evaporation) \
                        .addBands(img_wet) \
                        .clip(union_geometry)
                        
# 确定取样范围
samples = img_all.sampleRegion(union_geometry, 300, True)

# 在训练样本中增加一列随机数, 选取80%的样本为训练样本, 选取20%的样本为验证样本
sample = samples.randomColumn()
training_sample = sample.filter(aie.Filter.lte('random', 0.70))
validation_sample = sample.filter(aie.Filter.gt('random', 0.70))

In [ ]:
# 创建自适应集成分类器，并进行训练
trained_classifier = aie.Classifier.adaBoost(10)
trained_classifier = trained_classifier.train(training_sample, 
                                                 label, 
                                                 img_all.bandNames().getInfo())

# 获取训练样本的混淆矩阵, 并对训练精度进行评估
train_accuracy = trained_classifier.confusionMatrix()
print('Training error matrix:', train_accuracy.getInfo())
print('Training overall accuracy:', train_accuracy.accuracy().getInfo())


# 使用验证集对分类器进行评估
validation = validation_sample.classify(trained_classifier)
validation_accuracy = validation.errorMatrix(label, 'classification')
print('Validation error matrix:', validation_accuracy.getInfo())
print('Validation accuracy:', validation_accuracy.accuracy().getInfo())

# 使用训练好的分类器对影像进行分类
img_classified = img_all.classify(trained_classifier)


: 

In [ ]:
# 自适应集成分类图层
print(img_classified.getInfo())

map = aie.Map(
    center=union_fc.getCenter(),
    height=800,
    zoom=6
)

map.addLayer(
    img_classified,
    {
        'min': 0,
        'max': 1,
        'palette': ["0,255,255","255,0,0"]
    },
    'aie_classification',
    bounds=union_fc.getBounds()
)

map

# save(img_all, 'img_all')